In [123]:
# This is just a visual thing, I don't like how Jupyter usually displays things
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
}
"""

HTML('<style>{}</style>'.format(CSS))

# Data processing

In [122]:
import pandas as pd
PARSING_EN_TRAIN = 'DRS_parsing-master/parsing/layer_data/gold/en/dev.conll'
df = pd.DataFrame(columns = ["sentence", "word", "sym", "sem", "sns"])

def read_input(in_file):
    examples = []
    id = ''
    with open("data.txt") as f:
        for line in f.readlines():
            if '#' in line:
                id = line.strip().split('=')[1]
            elif len(line) > 1:
                sp = line.strip().split('\t')
                df.loc[len(df)] = [id, sp[0], sp[1], sp[2], sp[4]]
    return df

dat = read_input(PARSING_EN_TRAIN)
display(dat)

,sentence,word,sym,sem,sns
0,p00/d2862,Tom,tom,PER,male.n.02
1,p00/d2862,was,be,PST,O
2,p00/d2862,carrying,carry,EXG,carry.v.01
3,p00/d2862,a,a,DIS,O
4,p00/d2862,bucket,bucket,CON,bucket.n.01
...,...,...,...,...,...
5418,p10/d2454,.,.,NIL,O
5419,p10/d2335,Tom,tom,PER,male.n.02
5420,p10/d2335,was,be,PST,O
5421,p10/d2335,unarmed,unarmed,IST,unarmed.a.01


# Count occurrences of each word, respective to each word sense
### Make dataframe of all unique words and their total counts (not respective to sense)

In [102]:
unique_words = dat['word'].value_counts().rename_axis('unique_word').reset_index(name='total_count')
display(unique_words)

,unique_word,total_count
0,.,776
1,I,177
2,the,176
3,Tom,173
4,is,168
...,...,...
1557,talk,1
1558,begin,1
1559,bright,1
1560,rented,1


### Per unique word, get all unique senses it's used with and their counts.  Store in another dataframe. Also, store all ambiguous words in another dataframe. 

In [128]:
ambiguous_words = pd.DataFrame(columns = ['word', 'sense', 'count'])
word_df = pd.DataFrame(columns = ['word', 'sense', 'count'])

# Loop over unique words and get their unique senses
for word in unique_words['unique_word']:
    ambiguous = False
    word_dis = dat[dat['word'] == word]
    sense_counts = word_dis['sns'].value_counts().rename_axis('sense').reset_index(name='counts')
    if len(sense_counts) > 1:
        ambiguous = True
    
    # Store in dataframe
    for i, row in sense_counts.iterrows():
        word_df.loc[len(word_df)] = [word, row['sense'], row['counts']]
        if ambiguous:
            ambiguous_words.loc[len(ambiguous_words)] = [word, row['sense'], row['counts']]
            
display(word_df)
display(ambiguous_words)

,word,sense,count
0,.,O,776
1,I,O,177
2,the,O,176
3,Tom,male.n.02,172
4,Tom,O,1
...,...,...,...
1771,talk,talk.v.02,1
1772,begin,begin.v.01,1
1773,bright,bright.a.01,1
1774,rented,rent.v.03,1


,word,sense,count
0,Tom,male.n.02,172
1,Tom,O,1
2,is,O,125
3,is,be.v.02,19
4,is,be.v.01,12
...,...,...,...
366,grew,grow.v.07,1
367,table,table.n.01,1
368,table,table.n.02,1
369,clear,clear.a.01,1


In [127]:
# Some examples
display(word_df[word_df['word']=='Tom'])
display(word_df[word_df['word']=='is'])

,word,sense,count
3,Tom,male.n.02,172
4,Tom,O,1


,word,sense,count
5,is,O,125
6,is,be.v.02,19
7,is,be.v.01,12
8,is,be.v.08,4
9,is,be.v.03,4
10,is,be.v.06,3
11,is,be.v.13,1


# Naive Bayesian classifier